In [71]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import cdist
import random


In [4]:
cases_text_onlyspeaker_df = pd.read_csv("cases_text_onlyspeaker.csv")
cases_text_onlyspeaker_df.head()

,new_name,sentence,sentence_clean,topic_mentioned,sentiment,prob_score_neg,prob_score_neutral,prob_score_pos,has_parking,has_econactivity,...,sent_voice_mentioned,sentpol_voice_mentioned,prob_pos_voice_mentioned,prob_neg_voice_mentioned,positive_voice_mentioned,negative_voice_mentioned,num_voice_mentioned,num_sentences,sentence_lower,count_my_name
0,09-12-2017audio_6ZA-2017-210-CU-1A,My name is Jerry Newman.,My name is Jerry Newman.,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,my name is jerry newman.,1
1,09-12-2017audio_6ZA-2017-210-CU-1A,"Thank you, President Chunkham, members of the ...","Thank you, President Chunkham, members of the ...",NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,"thank you, president chunkham, members of the ...",0
2,09-12-2017audio_6ZA-2017-210-CU-1A,The question that is asked and that was asked ...,The question that is asked and that was asked ...,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,the question that is asked and that was asked ...,0
3,09-12-2017audio_6ZA-2017-210-CU-1A,And I think Mr. Chang made an adequate descrip...,And I think Mr. Chang made an adequate descrip...,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,and i think mr. chang made an adequate descrip...,0
4,09-12-2017audio_6ZA-2017-210-CU-1A,We started thinking through a business plan an...,We started thinking through a business plan an...,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,we started thinking through a business plan an...,0


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def get_embeddings(column):
  rows = cases_text_onlyspeaker_df[cases_text_onlyspeaker_df[column] == 1]
  all_sentences = rows["sentence"].tolist()
  
  return model.encode(all_sentences), all_sentences

In [7]:
#cosine similarity
parking_embeddings, parking_sentences = get_embeddings('has_parking')

# for i in range(len(parking_embeddings)):
  
# a = util.pytorch_cos_sim(embedding_1, embedding_2)

print(parking_embeddings[0].shape)

(384,)


In [55]:
sentences = [
    "Lacking space for parking in this current parking lot.",
    "We have no parking in this building.",
    "However, the first time you visit the restaurant, if you drive up and find parking in that neighborhood in Venice, good luck.",
    "We are very concerned about enough parking for our carseverybody to keep it off the already loaded with cars.",
    "And if youre a senator, then theres already more pressure on the parking in the neighborhood and keep in mind the other factor that theres absolutely no parking on Sunday.",
    "This is their plan to put parking and back of our building, leaving us with absolutely no parkingparking whatsoever and weve shared this parking for about 30 Ive been in the business for 31 years Ive been in that community for 31 years Thank you And is that all that I have one minute?",
    "And there is no parking.",
    "However, the first time you visit the restaurant, if you drive up and find parking in that neighborhood in Venice, good luck.",
    "So is there going to be, I think I, is there going to be any offside parking thats going to be designated for some of the people that work there?",
    "Again, that is a reason for the additional parking which is required in this case, which I previously explained.",
    "So Im strongly recommending that the residents create another parking district so we can have parking spaces in front of our homes because these on-site off-site parking that the restaurant applicants are suggestingwill not suffice.",
    "The neighborhood is already dealing with a two-and-a-half 25 capacity bar set to open this year with with very little to no parking and whose hours of operation would be from 11 amto 2 am.",
    "Theres not enough parking for guests and And there was one suggestion of the home on the site be up for harvesting."
]
example_sentence_embedding = model.encode(sentences)

counter_sentences = [
    "Furthermore, the Coastal Commission has denied us permit parking in our neighborhood.",
    "The certificate occupancy was issued condition upon the the recreation room being provided in the same way that a certificate occupancy is our issue condition upon parking being provided.",
    "They are asking deviation from the required parking for the fifth dwelling unit.",
    "Very clearly and as I stated earlier when looking at certificate occupancy and there is something that stated in a certificate occupancy that has to be provided and I relate to you when parking has to be provided.",
    "And we will try to make the, we will be adding the bike, the biking racks to help alleviate that extra parking.",
    "So what Dana is saying is, can we maybe amend any existing surface parking?",
    "But it is, it was built originally to allow parking there, and again, its interlocking favors.",
    "Thank youthat if they could have two security guards instead of three as requested in here in the parking lot versus in the restaurant, what would would that be adequate?",
    "Chains in the parking lot, their metal chains on plastic ballads.",
    "Many of the children walk to the school, they have to pass these parking lots, they have to pass these commercial zoned properties where cars are going in and out all the time.",
    "One of the things that attracted me to that space and to the bank building that is across the street from the Masonic lodge is the luxury of parking at night.",
    "Wheres the ADA parking spot, parking spot, plural or singular?"
]

counter_sentences_embeddings = model.encode(counter_sentences)


In [56]:
topic_sentence = ["lack parking", "no parking", "there is not enough parking", "needs to be more parking"]

topic_sentence_embedding = model.encode(topic_sentence)

In [57]:
for j in range(len(topic_sentence_embedding)):
  topic_scores_ls = []
  for i in range(len(example_sentence_embedding)):
    cos_similarity = util.pytorch_cos_sim(topic_sentence_embedding[j], example_sentence_embedding[i])
    cos_similarity_value = round(cos_similarity.item(), 2)
    topic_scores_ls.append(cos_similarity_value)

  counter_avg = sum(topic_scores_ls) / len(topic_scores_ls)

  print(f"{topic_sentence[j]}:  {counter_avg:.2f} avr, {topic_scores_ls}")

print("\n Counter-sentences:")
for j in range(len(topic_sentence_embedding)):
  topic_scores_ls = []
  for i in range(len(counter_sentences_embeddings)):
    cos_similarity = util.pytorch_cos_sim(topic_sentence_embedding[j], counter_sentences_embeddings[i])
    cos_similarity_value = round(cos_similarity.item(), 2)
    topic_scores_ls.append(cos_similarity_value)

  counter_avg = sum(topic_scores_ls) / len(topic_scores_ls)

  print(f"{topic_sentence[j]}:  {counter_avg:.2f} avr, {topic_scores_ls}")


lack parking:  0.54 avr, [0.74, 0.65, 0.34, 0.55, 0.5, 0.51, 0.72, 0.34, 0.47, 0.59, 0.48, 0.51, 0.58]
no parking:  0.54 avr, [0.64, 0.79, 0.41, 0.48, 0.48, 0.51, 0.83, 0.41, 0.47, 0.49, 0.47, 0.5, 0.54]
there is not enough parking:  0.54 avr, [0.76, 0.63, 0.34, 0.57, 0.48, 0.51, 0.72, 0.34, 0.47, 0.63, 0.49, 0.49, 0.58]
needs to be more parking:  0.53 avr, [0.75, 0.48, 0.36, 0.6, 0.54, 0.54, 0.59, 0.36, 0.51, 0.59, 0.55, 0.52, 0.55]

 Counter-sentences:
lack parking:  0.42 avr, [0.37, 0.35, 0.49, 0.4, 0.48, 0.45, 0.44, 0.34, 0.31, 0.48, 0.39, 0.51]
no parking:  0.39 avr, [0.43, 0.33, 0.39, 0.41, 0.45, 0.46, 0.41, 0.28, 0.29, 0.41, 0.37, 0.5]
there is not enough parking:  0.41 avr, [0.38, 0.38, 0.5, 0.39, 0.44, 0.45, 0.46, 0.35, 0.23, 0.45, 0.34, 0.5]
needs to be more parking:  0.45 avr, [0.35, 0.36, 0.52, 0.41, 0.61, 0.55, 0.49, 0.43, 0.3, 0.5, 0.41, 0.51]


In [73]:
def find_similar_sentences(topic_sentences, sentence_embeddings, sentences, threshold=0.6):
    similar_sentences = []
    not_similar_sentences = []
    topic_sentence_embedding = model.encode(topic_sentences)
    
    for i, sentence_embedding in enumerate(sentence_embeddings):
        cos_similarity_ls = [
            util.pytorch_cos_sim(sentence_embedding, topic_embedding).item()
            for topic_embedding in topic_sentence_embedding
        ]
        
        if any(val > threshold for val in cos_similarity_ls):
            similar_sentences.append(sentences[i])
        else:
            not_similar_sentences.append(sentences[i])
    
    return similar_sentences, not_similar_sentences


In [76]:
topic_sentence = ["lack parking", "no parking", "there is not enough parking", "needs to be more parking"]

threshold = 0.55
similar_sentences, not_similar_sentences = find_similar_sentences(topic_sentence, parking_embeddings, parking_sentences, threshold)


print("Random sample of 10 similar sentences:")
for sentence in random.sample(similar_sentences, min(10, len(similar_sentences))):
    print(sentence)
print(f"\nTotal number of similar sentences: {len(similar_sentences)}")

print("\nRandom sample of 10 not similar sentences:")
for sentence in random.sample(not_similar_sentences, min(10, len(not_similar_sentences))):
    print(sentence)
print(f"\nTotal number of not similar sentences: {len(not_similar_sentences)}")

Random sample of 10 similar sentences:
He basically submitted an application and said he needed no additional parking.
So that might be an area that youd be able to utilize and try and reduce that number of people that use the parking even though there may be where large trucks for production are using the parking and theres not actually those vehicles miles traveled in the area, but it might help offset that number youre kind of looking at.
I just want to kind of recap what we are parking in that parking lot.
Okay, so your parking is none of it is sub-training.
Theres parking.
So theyre going to be parking in our neighborhood.
Okayand somehow designating where the parking entrance is.
I own a small, I run a small corner in the market and theres been always been a big issue with the parking and the create town is getting more and more crowded and Im glad this projects coming in its gonna provide Im a hundred of parking there, especially with bicycle parking.
All right comments are just

In [ ]:
#could implement exclusivity. 4 topic, match to most similar one if each above threshold
#add accuracy